In [ ]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [ ]:
PUZZLE_DIM = 5
action = namedtuple('Action', ['pos1', 'pos2'])

In [ ]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [9]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal = state
print(state)

[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24  0]]


In [10]:
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[19, 13,  2,  9, 24],
       [18,  7,  5, 22,  8],
       [10, 23,  0, 15,  3],
       [16, 14, 20, 17, 12],
       [ 4,  6, 11,  1, 21]])

TO DO LIST:
- implement the algorithm to find the sequence of action , we have to order the matrix

- create a matrix that is a mask, apply the algorithm, check if the solution can be good
-apply first a path search


In [ ]:
sequence = []

In [ ]:
# state = do_action(state, choice(available_actions(state)))

while state!=goal:
